In [1]:
from __future__ import print_function

from miscc.utils import mkdir_p
from miscc.utils import build_super_images
from miscc.losses import sent_loss, words_loss
from miscc.config import cfg, cfg_from_file

from datasets import TextDataset
from datasets import prepare_data

from model import RNN_ENCODER, CNN_ENCODER

import os
import sys
import time
import random
import pprint
import datetime
import dateutil.tz
import argparse
import numpy as np
from PIL import Image

import torch
import torch.nn as nn
import torch.optim as optim
from torchsummary import summary
from torch.autograd import Variable
import torch.backends.cudnn as cudnn
import torchvision.transforms as transforms

In [2]:
# dataset.n_words

In [3]:
cfg.TEXT.EMBEDDING_DIM

256

In [4]:
def build_models():
    # build model ############################################################
    text_encoder = RNN_ENCODER(dataset.n_words, nhidden=cfg.TEXT.EMBEDDING_DIM)
    cfg.TREE.BASE_SIZE = torch.FloatTensor(299)
#     image_encoder = CNN_ENCODER(cfg.TREE.BASE_SIZE) #299 inside CNN_Encoder
    labels = torch.LongTensor(range(batch_size))
    start_epoch = 0
    if cfg.TRAIN.NET_E != '':
        state_dict = torch.load(cfg.TRAIN.NET_E)
        text_encoder.load_state_dict(state_dict)
        print('Load ', cfg.TRAIN.NET_E)
        #
        name = cfg.TRAIN.NET_E.replace('text_encoder')
        state_dict = torch.load(name)
#         image_encoder.load_state_dict(state_dict)
        print('Load ', name)

        istart = cfg.TRAIN.NET_E.rfind('_') + 8
        iend = cfg.TRAIN.NET_E.rfind('.')
        start_epoch = cfg.TRAIN.NET_E[istart:iend]
        start_epoch = int(start_epoch) + 1
        print('start_epoch', start_epoch)
    if cfg.CUDA:
        text_encoder = text_encoder
#         image_encoder = image_encoder
        labels = labels

    return text_encoder, labels, start_epoch

In [5]:
cfg.TEXT.EMBEDDING_DIM

256

In [6]:
print(torch.__version__)

0.4.0


In [7]:
#Configurations
cfg_from_file("cfg/DAMSM/coco.yml")
import pprint
pprint.pprint(cfg)
random.seed(random.randint(1, 10000))
np.random.seed(random.randint(1, 10000))
torch.manual_seed(random.randint(1, 10000))
if cfg.CUDA:
    torch.cuda.manual_seed_all(random.randint(1, 10000))
now = datetime.datetime.now(dateutil.tz.tzlocal())
timestamp = now.strftime('%Y_%m_%d_%H_%M_%S')
output_dir = '../output/%s_%s_%s' % \
    (cfg.DATASET_NAME, cfg.CONFIG_NAME, timestamp)

model_dir = os.path.join(output_dir, 'Model')
image_dir = os.path.join(output_dir, 'Image')
mkdir_p(model_dir)
mkdir_p(image_dir)
# Debug
torch.cuda.device(cfg.GPU_ID)
cudnn.benchmark = True

{'B_VALIDATION': False,
 'CONFIG_NAME': 'DAMSM',
 'CUDA': True,
 'DATASET_NAME': 'coco',
 'DATA_DIR': '../data/coco',
 'GAN': {'B_ATTENTION': True,
         'B_DCGAN': False,
         'CONDITION_DIM': 100,
         'DF_DIM': 64,
         'GF_DIM': 128,
         'R_NUM': 2,
         'Z_DIM': 100},
 'GPU_ID': 0,
 'RNN_TYPE': 'LSTM',
 'TEXT': {'CAPTIONS_PER_IMAGE': 5, 'EMBEDDING_DIM': 256, 'WORDS_NUM': 15},
 'TRAIN': {'BATCH_SIZE': 48,
           'B_NET_D': True,
           'DISCRIMINATOR_LR': 0.0002,
           'ENCODER_LR': 0.002,
           'FLAG': True,
           'GENERATOR_LR': 0.0002,
           'MAX_EPOCH': 600,
           'NET_E': '',
           'NET_G': '',
           'RNN_GRAD_CLIP': 0.25,
           'SMOOTH': {'GAMMA1': 4.0,
                      'GAMMA2': 5.0,
                      'GAMMA3': 10.0,
                      'LAMBDA': 1.0},
           'SNAPSHOT_INTERVAL': 5},
 'TREE': {'BASE_SIZE': 299, 'BRANCH_NUM': 1},
 'WORKERS': 1}


In [8]:
# cfg.TREE.BASE_SIZE = 299
imsize = cfg.TREE.BASE_SIZE * (2 ** (cfg.TREE.BRANCH_NUM-1))
batch_size = cfg.TRAIN.BATCH_SIZE
image_transform = transforms.Compose([
    transforms.Scale(int(imsize * 76 / 64)),
    transforms.RandomCrop(imsize),
    transforms.RandomHorizontalFlip()])
dataset = TextDataset("../data/coco", 'train',
                      base_size=299,
                      transform=image_transform)
print("=====================================================")
print("cfg.DATA_DIR: ", cfg.DATA_DIR)
print("cfg.TREE.BASE_SIZE: ", cfg.TREE.BASE_SIZE)
print("image_transform: ", image_transform)
print("cfg.TEXT.EMBEDDING_DIM", cfg.TEXT.EMBEDDING_DIM)
print("=====================================================")
print("dataset.n_words:", dataset.n_words, " dataset.embeddings_num: ", dataset.embeddings_num)
assert dataset
dataloader = torch.utils.data.DataLoader(
    dataset, batch_size=batch_size, drop_last=True,
    shuffle=True, num_workers=int(cfg.WORKERS))

# # validation data #
dataset_val = TextDataset("../data/coco", 'test',
                          base_size=cfg.TREE.BASE_SIZE,
                          transform=image_transform)
dataloader_val = torch.utils.data.DataLoader(
    dataset_val, batch_size=batch_size, drop_last=True,
    shuffle=True, num_workers=int(cfg.WORKERS))

Load filenames from: ../data/coco/train/filenames.pickle (82783)
Load filenames from: ../data/coco/test/filenames.pickle (40470)
Load from:  ../data/coco/captions.pickle
cfg.DATA_DIR:  ../data/coco
cfg.TREE.BASE_SIZE:  299
image_transform:  <torchvision.transforms.Compose object at 0x1c1eb064a8>
cfg.TEXT.EMBEDDING_DIM 256
dataset.n_words: 27297  dataset.embeddings_num:  5
Load filenames from: ../data/coco/train/filenames.pickle (82783)
Load filenames from: ../data/coco/test/filenames.pickle (40470)
Load from:  ../data/coco/captions.pickle


In [9]:
split_dir, bshuffle = 'train', True
if not cfg.TRAIN.FLAG:
    # bshuffle = False
    split_dir = 'test'
dataset = TextDataset(cfg.DATA_DIR, 'train',base_size=cfg.TREE.BASE_SIZE,transform=image_transform)
dataloader = torch.utils.data.DataLoader(dataset, batch_size=cfg.TRAIN.BATCH_SIZE,drop_last=True, shuffle=bshuffle, num_workers=int(cfg.WORKERS))

Load filenames from: ../data/coco/train/filenames.pickle (82783)
Load filenames from: ../data/coco/test/filenames.pickle (40470)
Load from:  ../data/coco/captions.pickle


In [10]:
dataset

In [11]:
text_encoder, labels, start_epoch = build_models()

/Users/user/anaconda/lib/python3.6/site-packages/torch/nn/modules/rnn.py:38: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.5 and num_layers=1
  "num_layers={}".format(dropout, num_layers))


In [12]:
text_encoder.train()
text_encoder.eval()

RNN_ENCODER(
  (encoder): Embedding(27297, 300)
  (drop): Dropout(p=0.5)
  (rnn): LSTM(300, 128, batch_first=True, dropout=0.5, bidirectional=True)
)

In [13]:
for step, data in enumerate(dataloader, 0):
    imgs, captions, cap_lens, class_ids, keys = prepare_data(data)
    hidden = text_encoder.init_hidden(batch_size)
    print("step", step, "\ncap_lens\n", cap_lens, "\n") 
    if step == 0:
        break

step 0 
cap_lens
 tensor([ 15,  15,  15,  15,  14,  14,  14,  13,  13,  13,  12,  11,
         11,  11,  11,  11,  11,  10,  10,  10,  10,  10,  10,  10,
         10,  10,  10,  10,  10,  10,   9,   9,   9,   9,   9,   9,
          9,   9,   9,   8,   8,   8,   8,   8,   8,   8,   8,   8]) 



In [14]:
print(captions.type())
print(captions.size())
print(captions)

torch.LongTensor
torch.Size([48, 15])
tensor([[ 23046,  15790,   4114,   1943,  23046,   2861,  20752,  26444,
          13598,   7515,  23046,   7510,  13540,   1943,  10402],
        [ 23046,   8494,  26444,  16860,  11824,  10409,  23484,  23046,
          15298,  15788,  10400,   1327,  14435,  23046,  14018],
        [ 23046,  18903,  25218,  15626,   4114,   1943,  23046,  13453,
           1327,  14435,  23046,    822,  23046,   8684,   9735],
        [ 23046,   6842,   1476,   7510,   7169,  10409,   8707,  26007,
            822,  23046,    431,  14309,   8707,   3823,  23537],
        [ 23046,  12441,   8030,   1327,  14435,  23046,  17528,   2667,
           1327,  14435,  23046,  17732,  12713,  15400,      0],
        [  7510,  20793,  15618,  12713,  23046,  12791,  25218,  23046,
          17167,    822,  24669,   7515,  23046,  18044,      0],
        [ 23046,  14314,   9294,  15788,  21337,   3638,  13949,  12713,
           7510,   5053,  10977,  26444,  13598,  14803

In [15]:
# captions_dummy_input = torch.rand(48, 15, requires_grad=True)
captions_fake_input = torch.randint(1, 23046, (48, 15), dtype=torch.long)
# captions_dummy_input = torch.rand(48, 15, requires_grad=True)
print(captions_fake_input.size())
print(captions_fake_input)

torch.Size([48, 15])
tensor([[ 11668,  12098,   2662,  15455,  15185,  10306,   6496,  22388,
            366,    283,  14938,   3638,  18888,   2598,  21660],
        [ 12028,   5654,  13752,  21115,   3584,  11413,  16984,   9818,
           2144,   4163,   8200,   3056,  18869,   8373,  16521],
        [ 17606,     58,    870,   3119,  15609,  20938,  15537,  18282,
          20109,   5292,  17121,  16849,   4809,   4832,  10013],
        [  7315,  16826,  20212,  16158,   1352,  18028,   9202,  13897,
          15043,  19434,  14067,   3237,  13822,  11535,   6125],
        [ 10124,   1259,   6210,   2046,   7866,  21490,  14073,  14276,
          12912,   9276,  16102,   4282,  11358,  19185,  17393],
        [  4033,   6821,  21362,  21832,  23001,  10885,  21415,   6851,
           2076,   5379,  21921,   8077,   1631,  10898,  11493],
        [  3971,   2381,  10953,   5514,   4727,  17529,   3354,   9348,
          17886,  13089,   7718,  16071,  18400,  19825,  16861],
      

In [16]:
print(cap_lens.type())
print(cap_lens.size())
print(cap_lens)

torch.LongTensor
torch.Size([48])
tensor([ 15,  15,  15,  15,  14,  14,  14,  13,  13,  13,  12,  11,
         11,  11,  11,  11,  11,  10,  10,  10,  10,  10,  10,  10,
         10,  10,  10,  10,  10,  10,   9,   9,   9,   9,   9,   9,
          9,   9,   9,   8,   8,   8,   8,   8,   8,   8,   8,   8])


In [17]:
# cap_lens_fake_input = torch.rand(48, dtype=torch.float64)
cap_lens_fake_input = torch.randint(1, 15, (48,), dtype=torch.long)
# cap_lens_dummy_input = torch.FloatTensor(cap_lens_dummy_input)
print(cap_lens_fake_input.size())
print(cap_lens_fake_input)

torch.Size([48])
tensor([ 13,   7,   6,  13,  14,   7,   6,   8,   4,  14,   1,  14,
         12,   4,   6,   5,   6,   8,   5,   5,   7,   9,   2,  11,
          3,  13,  12,  13,   1,  12,   6,   9,  12,  11,  12,  14,
         13,  13,   7,   2,   2,   9,   7,   2,   2,   3,   4,   6])


In [18]:
print("hidden[0].shape: ", hidden[0].shape, " and data type: ", hidden[0].type())
print("hidden[1].shape: ", hidden[1].shape, " and data type: ", hidden[1].type())
print(hidden)

hidden[0].shape:  torch.Size([2, 48, 128])  and data type:  torch.FloatTensor
hidden[1].shape:  torch.Size([2, 48, 128])  and data type:  torch.FloatTensor
(tensor([[[ 0.,  0.,  0.,  ...,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  ...,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  ...,  0.,  0.,  0.],
         ...,
         [ 0.,  0.,  0.,  ...,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  ...,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  ...,  0.,  0.,  0.]],

        [[ 0.,  0.,  0.,  ...,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  ...,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  ...,  0.,  0.,  0.],
         ...,
         [ 0.,  0.,  0.,  ...,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  ...,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  ...,  0.,  0.,  0.]]]), tensor([[[ 0.,  0.,  0.,  ...,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  ...,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  ...,  0.,  0.,  0.],
         ...,
         [ 0.,  0.,  0.,  ...,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  ...,  0.,  0.,  0.

In [19]:
# hidden_fake_input = torch.randint(0, 1 (2, 48, 128), dtype=torch.long)
hidden_fake_input = torch.randint(1, 10, (2, 48, 128), dtype=torch.long)
# hidden_fake_input = torch.FloatTensor(hidden_fake_input)
print(hidden_fake_input.size())
print(hidden_fake_input)

torch.Size([2, 48, 128])
tensor([[[ 8,  8,  5,  ...,  1,  5,  4],
         [ 8,  4,  3,  ...,  2,  1,  9],
         [ 1,  1,  2,  ...,  1,  8,  3],
         ...,
         [ 3,  2,  3,  ...,  3,  5,  3],
         [ 3,  4,  3,  ...,  3,  9,  1],
         [ 7,  3,  1,  ...,  7,  3,  7]],

        [[ 3,  8,  1,  ...,  4,  3,  8],
         [ 1,  2,  7,  ...,  9,  6,  8],
         [ 5,  1,  6,  ...,  5,  6,  5],
         ...,
         [ 7,  8,  3,  ...,  9,  8,  7],
         [ 6,  4,  2,  ...,  1,  3,  1],
         [ 5,  6,  4,  ...,  9,  2,  9]]])


In [20]:
# output = text_encoder(captions, cap_lens, hidden)
words_emb, sent_emb = text_encoder(captions, cap_lens, hidden)

r = (captions, cap_lens, hidden)

In [21]:
r

(tensor([[ 23046,  15790,   4114,   1943,  23046,   2861,  20752,  26444,
           13598,   7515,  23046,   7510,  13540,   1943,  10402],
         [ 23046,   8494,  26444,  16860,  11824,  10409,  23484,  23046,
           15298,  15788,  10400,   1327,  14435,  23046,  14018],
         [ 23046,  18903,  25218,  15626,   4114,   1943,  23046,  13453,
            1327,  14435,  23046,    822,  23046,   8684,   9735],
         [ 23046,   6842,   1476,   7510,   7169,  10409,   8707,  26007,
             822,  23046,    431,  14309,   8707,   3823,  23537],
         [ 23046,  12441,   8030,   1327,  14435,  23046,  17528,   2667,
            1327,  14435,  23046,  17732,  12713,  15400,      0],
         [  7510,  20793,  15618,  12713,  23046,  12791,  25218,  23046,
           17167,    822,  24669,   7515,  23046,  18044,      0],
         [ 23046,  14314,   9294,  15788,  21337,   3638,  13949,  12713,
            7510,   5053,  10977,  26444,  13598,  14803,      0],
         [ 17

In [22]:
print(words_emb.shape)
print(words_emb)

torch.Size([48, 256, 15])
tensor([[[-0.0164, -0.0309, -0.0140,  ..., -0.0535, -0.0200, -0.0187],
         [-0.0042,  0.0009,  0.0209,  ...,  0.0270,  0.0321,  0.0131],
         [-0.0017,  0.0320,  0.0279,  ...,  0.0503,  0.0445,  0.0481],
         ...,
         [ 0.0087,  0.0065,  0.0439,  ..., -0.0036,  0.0153,  0.0045],
         [ 0.0239,  0.0305,  0.0259,  ...,  0.0180,  0.0014,  0.0068],
         [ 0.0138, -0.0134, -0.0073,  ...,  0.0154, -0.0035, -0.0088]],

        [[-0.0164, -0.0059,  0.0147,  ..., -0.0279, -0.0409, -0.0557],
         [-0.0042, -0.0241,  0.0050,  ...,  0.0065, -0.0015, -0.0041],
         [-0.0017, -0.0030,  0.0245,  ...,  0.0342,  0.0207,  0.0225],
         ...,
         [-0.0022, -0.0182,  0.0177,  ...,  0.0144,  0.0124,  0.0122],
         [ 0.0201,  0.0294,  0.0232,  ..., -0.0077,  0.0132,  0.0022],
         [ 0.0231, -0.0021, -0.0391,  ...,  0.0383,  0.0098, -0.0297]],

        [[-0.0164,  0.0098, -0.0241,  ..., -0.0496, -0.0231, -0.0433],
         [-0.0042, 

In [23]:
print(sent_emb.shape)
print(sent_emb)

torch.Size([48, 256])
tensor([[-1.8697e-02,  1.3067e-02,  4.8063e-02,  ...,  8.6733e-03,
          2.3882e-02,  1.3803e-02],
        [-5.5712e-02, -4.1268e-03,  2.2523e-02,  ..., -2.2310e-03,
          2.0061e-02,  2.3147e-02],
        [-4.3290e-02,  3.8086e-02,  6.3508e-02,  ...,  6.8706e-03,
          1.4765e-02,  1.2867e-02],
        ...,
        [-2.8123e-02,  1.6622e-02,  5.8342e-02,  ...,  2.1916e-02,
          8.4493e-03, -2.3437e-02],
        [-6.6720e-02,  1.4136e-02,  4.1421e-02,  ...,  2.1916e-02,
          1.3408e-02,  1.0613e-02],
        [-3.0940e-02, -9.0442e-03,  4.4092e-02,  ..., -1.0927e-03,
          1.3184e-02,  2.0895e-02]])


In [24]:
# source coreml/bin/activate
import coremltools

In [25]:
# output = text_encoder(captions, cap_lens, hidden)
captions_fake_input = torch.randint(1, 23046, (48, 15), dtype=torch.long)

In [26]:
words_emb_, sent_emb_ = text_encoder(captions, cap_lens, hidden)

In [27]:
words_emb_

tensor([[[-0.0164, -0.0309, -0.0140,  ..., -0.0535, -0.0200, -0.0187],
         [-0.0042,  0.0009,  0.0209,  ...,  0.0270,  0.0321,  0.0131],
         [-0.0017,  0.0320,  0.0279,  ...,  0.0503,  0.0445,  0.0481],
         ...,
         [ 0.0087,  0.0065,  0.0439,  ..., -0.0036,  0.0153,  0.0045],
         [ 0.0239,  0.0305,  0.0259,  ...,  0.0180,  0.0014,  0.0068],
         [ 0.0138, -0.0134, -0.0073,  ...,  0.0154, -0.0035, -0.0088]],

        [[-0.0164, -0.0059,  0.0147,  ..., -0.0279, -0.0409, -0.0557],
         [-0.0042, -0.0241,  0.0050,  ...,  0.0065, -0.0015, -0.0041],
         [-0.0017, -0.0030,  0.0245,  ...,  0.0342,  0.0207,  0.0225],
         ...,
         [-0.0022, -0.0182,  0.0177,  ...,  0.0144,  0.0124,  0.0122],
         [ 0.0201,  0.0294,  0.0232,  ..., -0.0077,  0.0132,  0.0022],
         [ 0.0231, -0.0021, -0.0391,  ...,  0.0383,  0.0098, -0.0297]],

        [[-0.0164,  0.0098, -0.0241,  ..., -0.0496, -0.0231, -0.0433],
         [-0.0042,  0.0105, -0.0012,  ...,  0

In [28]:
sent_emb_

tensor([[-1.8697e-02,  1.3067e-02,  4.8063e-02,  ...,  8.6733e-03,
          2.3882e-02,  1.3803e-02],
        [-5.5712e-02, -4.1268e-03,  2.2523e-02,  ..., -2.2310e-03,
          2.0061e-02,  2.3147e-02],
        [-4.3290e-02,  3.8086e-02,  6.3508e-02,  ...,  6.8706e-03,
          1.4765e-02,  1.2867e-02],
        ...,
        [-2.8123e-02,  1.6622e-02,  5.8342e-02,  ...,  2.1916e-02,
          8.4493e-03, -2.3437e-02],
        [-6.6720e-02,  1.4136e-02,  4.1421e-02,  ...,  2.1916e-02,
          1.3408e-02,  1.0613e-02],
        [-3.0940e-02, -9.0442e-03,  4.4092e-02,  ..., -1.0927e-03,
          1.3184e-02,  2.0895e-02]])

In [29]:
text_encoder.train()

RNN_ENCODER(
  (encoder): Embedding(27297, 300)
  (drop): Dropout(p=0.5)
  (rnn): LSTM(300, 128, batch_first=True, dropout=0.5, bidirectional=True)
)

In [30]:
# captions_fake_input = captions_fake_input.float()
cap_lens = cap_lens.float()

In [31]:
captions_fake_input.type()

'torch.LongTensor'

In [32]:
# Export the model
torch_out = torch.onnx.export(text_encoder,                 # model being run
                               (captions_fake_input, cap_lens, hidden), # model input (or a tuple for multiple inputs)
                               "kol.onnx",      # where to save the model (can be a file or file-like object)
                               export_params=True)           # store the trained parameter weights inside the model file

TypeError: wrapPyFuncWithSymbolic(): incompatible function arguments. The following argument types are supported:
    1. (self: torch._C.Graph, arg0: function, arg1: List[torch::jit::Value], arg2: int, arg3: function) -> iterator

Invoked with: graph(%0 : Long(48, 15)
      %1 : Float(48)
      %2 : Float(2, 48, 128)
      %3 : Float(2, 48, 128)
      %4 : Float(27297, 300)
      %5 : Float(512, 300)
      %6 : Float(512, 128)
      %7 : Float(512)
      %8 : Float(512)
      %9 : Float(512, 300)
      %10 : Float(512, 128)
      %11 : Float(512)
      %12 : Float(512)) {
  %13 : Float(48, 15, 300) = aten::embedding[padding_idx=-1, scale_grad_by_freq=0, sparse=0](%4, %0), scope: RNN_ENCODER/Embedding[encoder]
  %16 : Float(48, 15, 300), %17 : Handle = ^Dropout(0.5, False, False)(%13), scope: RNN_ENCODER/Dropout[drop]
  %15 : Float(48, 15, 300) = aten::slice[dim=0, start=0, end=9223372036854775807, step=1](%13), scope: RNN_ENCODER/Dropout[drop]
  %14 : Float(48, 15, 300) = aten::as_stri